# The Solid Design Principle

## Single Responsibility Principle , Separation of concerns 

If you have a class it should have a primary responsibility and it should not take up other responsibility

In [1]:
class Journal:
    def __init__(self):
        self.entries = []
        self.count=0
    
    def add_entry(self,text):
        self.count +=1
        self.entries.append(f'{self.count}:{text}')
        
    def remove_entry(self,pos):
        del self.entries[pos]
        
    def __str__(self):
        return '\n'.join(self.entries)
    
j = Journal()
j.add_entry('Today is monday')
j.add_entry('I have to go to sleep')

print(f'Journal entries:\n {j}')

Journal entries:
 1:Today is monday
2:I have to go to sleep


- Lets break the Single Responsibility Principle, and give additional work to the Journal class , by asking it to save the Journal to a file.
- Add secondary responsiblity , now we are asking it to persist.
- Any change in the persistance will have to change in all the classes 

In [2]:
class Journal:
    def __init__(self):
        self.entries = []
        self.count=0
    
    def add_entry(self,text):
        self.count +=1
        self.entries.append(f'{self.count}:{text}')
        
    def remove_entry(self,pos):
        del self.entries[pos]
        
    def __str__(self):
        return '\n'.join(self.entries)
    
    ## Add secondary responsiblity , now we are asking it to persist.
    ## Any change in the pers
    def save(self,filename):
        file = open(filename,'w')
        file.write(srt(self))
        file.close()
    
    def load(self,filename):
        pass
    
j = Journal()
j.add_entry('Today is monday')
j.add_entry('I have to go to sleep')

print(f'Journal entries:\n {j}')

Journal entries:
 1:Today is monday
2:I have to go to sleep


To fix this issue we break the code and put it into another class called Persistance manager
This is to not create a anti-pattern, and avoid creating a God object . Not to put everything into one class.
A class should always have a single reason to change and that should be related to it primary responsibility.

In [3]:
class Journal:
    def __init__(self):
        self.entries = []
        self.count=0
    
    def add_entry(self,text):
        self.count +=1
        self.entries.append(f'{self.count}:{text}')
        
    def remove_entry(self,pos):
        del self.entries[pos]
        
    def __str__(self):
        return '\n'.join(self.entries)
    
    ## Add secondary responsiblity , now we are asking it to persist.
    ## Any change in the persitance will have to chage all the classes 
#     def save(self,filename):
#         file = open(filename,'w')
#         file.write(srt(self))
#         file.close()
    
#     def load(self,filename):
#         pass
    
class PersistanceManager:
    @staticmethod
    def save_to_file(journal,filename):
        file = open(filename,'w')
        file.write(str(journal))
        file.close()
    
j = Journal()
j.add_entry('Today is monday')
j.add_entry('I have to go to sleep')

print(f'Journal entries:\n {j}')

file = r'.\Journal.txt'
PersistanceManager.save_to_file(j,file)

Journal entries:
 1:Today is monday
2:I have to go to sleep


In [4]:
with open(file) as fh:
    print(fh.read())

1:Today is monday
2:I have to go to sleep


## Open Closed Principle 

This design Principle suggests, Classes should be "Open for Extension , but closed for modification"

In [7]:
from enum import Enum 

class Color(Enum):
    Red=1
    Green=2
    BLue=3 
    
class Size(Enum):
    Small=1
    Medium=2
    Large=3



class Product:
    def __init__(self,name , color, size):
        self.name=name
        self.color=color
        self.size=size

## Open for Extension , but closed for modification.
class ProductFilter:
    
    def filter_by_color(self,products,color):
        for p in products:
            if p.color==color : yield p
                
    def filter_by_size(self,products,size):
        for p in products:
            if p.size==size : yield p
                

Now if the requirement is to filter by size and color , we have to add anther method , which is like disobeying OCP
This Approch does not scale and we are causing state space explosion.
EG : Filter by size , filter by color , filter by size and color , filer by size or color 
If you have three criteria, then we have 7 methods to satisfy this case.

In [18]:
from enum import Enum 

class Color(Enum):
    Red=1
    Green=2
    Blue=3 
    
class Size(Enum):
    Small=1
    Medium=2
    Large=3



class Product:
    def __init__(self,name , color, size):
        self.name=name
        self.color=color
        self.size=size

## Open for Extension , but closed for modification.
class ProductFilter:
    
    def filter_by_color(self,products,color):
        for p in products:
            if p.color==color : yield p
                
    def filter_by_size(self,products,size):
        for p in products:
            if p.size==size : yield p
                
    def filter_by_size_and_color(self,products,size,color):
        for p in products:
            if p.size==size and p.color==color : 
                yield p
                
    def filter_by_size_or_color(self,products,size,color):
        for p in products:
            if p.size==size or p.color==color : 
                yield p

How to make sure we dont break the OPC , and this can be achieved by a Enterpirse pattern called specification.

Specification : Is a class which determines a particular item satisfy a particular criterian.

In [40]:
class Specification:
    def is_statisfied(self,item):#this method is to be overridden
        pass

class Filter:
    def filter(self,item,spec):
        pass

class ColorSpecification(Specification):
    
    def __init__(self,color):
        self.color=color
        
    def is_statified(self,item):
        return item.color==self.color
    
class SizeSpecification(Specification):
    
    def __init__(self,size):
        self.size=size
        
    def is_statisfied(self,item):
        return item.size==self.size
    
class Betterfilter(Filter):
    def filter(self,items,spec):
        #print("spec",spec)
        for item in items:
            if spec.is_statisfied(item):
                yield item

apple = Product('Apple',Color.Green,Size.Small)
tree = Product('Tree',Color.Green,Size.Large)
house = Product('House',Color.Blue,Size.Large)
    
products = [apple,tree,house]
    
pf = ProductFilter()
print('Green Product (old):')
 
for p in pf.filter_by_color(products,Color.Green):
    print(f'-{p.name} is green')
    
## Specification approach New
bf = Betterfilter()

print('Green Product (new):')
green=ColorSpecification(Color.Green)

for p in bf.filter(products,green):
    print(f'-{p.name} is green')

    
print('Large Product (new):')
large=SizeSpecification(Size.Large)

for p in bf.filter(products,large):
    print(f'-{p.name} is large')

Green Product (old):
-Apple is green
-Tree is green
Green Product (new):
Large Product (new):
-Tree is large
-House is large


Lets apply the Design Principle and create a and filter 

In [36]:
class Specification:
    def is_statisfied(self,item):#this method is to be overriddren
        pass

class Filter:
    def filter(self,item,spec):
        pass

class ColorSpecification(Specification):
    
    def __init__(self,color):
        self.color=color
        
    def is_statisfied(self,item):
        return item.color==self.color
    
class SizeSpecification(Specification):
    
    def __init__(self,size):
        self.size=size
        
    def is_statisfied(self,item):
        return item.size==self.size

class AndSpecification(Specification):
    def __init__(self,*arg):
        self.args=arg
                
    def is_statisfied(self,item):
        return all(map(lambda spec: spec.is_statisfied(item),self.args))
        
class Betterfilter(Filter):
    def filter(self,items,spec):
        for item in items:
            if spec.is_statisfied(item):
                yield item

apple = Product('Apple',Color.Green,Size.Small)
tree = Product('Tree',Color.Green,Size.Large)
house = Product('House',Color.Blue,Size.Large)
    
products = [apple,tree,house]
    
pf = ProductFilter()
print('Green Product (old):')
 
for p in pf.filter_by_color(products,Color.Green):
    print(f'-{p.name} is green')
    
## Specification approach New
bf = Betterfilter()

print('Green Product (new):')
green=ColorSpecification(Color.Green)

for p in bf.filter(products,green):
    print(f'-{p.name} is green')

    
print('Large Product (new):')
large=SizeSpecification(Size.Large)

for p in bf.filter(products,large):
    print(f'-{p.name} is large')
    
print('Large Blue Product (new):')
large_blue=AndSpecification(large,ColorSpecification(Color.Blue))

for p in bf.filter(products,large_blue):
    print(f'-{p.name} is large and blue')

Green Product (old):
-Apple is green
-Tree is green
Green Product (new):
-Apple is green
-Tree is green
Large Product (new):
-Tree is large
-House is large
Large Blue Product (new):
-House is large and blue


It avoid changing the exsisting code but allows extending the functionality

In [43]:
class Specification:
    def is_statisfied(self,item):#this method is to be overriddren
        pass
    ## overload and operator
    def __and__(self,other):
        return AndSpecification(self,other)

class Filter:
    def filter(self,item,spec):
        pass

class ColorSpecification(Specification):
    
    def __init__(self,color):
        self.color=color
        
    def is_statisfied(self,item):
        return item.color==self.color
    
class SizeSpecification(Specification):
    
    def __init__(self,size):
        self.size=size
        
    def is_statisfied(self,item):
        return item.size==self.size

class AndSpecification(Specification):
    def __init__(self,*arg):
        self.args=arg
                
    def is_statisfied(self,item):
        return all(map(lambda spec: spec.is_statisfied(item),self.args))
        
class Betterfilter(Filter):
    def filter(self,items,spec):
        for item in items:
            if spec.is_statisfied(item):
                yield item

apple = Product('Apple',Color.Green,Size.Small)
tree = Product('Tree',Color.Green,Size.Large)
house = Product('House',Color.Blue,Size.Large)
    
products = [apple,tree,house]
    
pf = ProductFilter()
print('Green Product (old):')
 
for p in pf.filter_by_color(products,Color.Green):
    print(f'-{p.name} is green')
    
## Specification approach New
bf = Betterfilter()

print('Green Product (new):')
green=ColorSpecification(Color.Green)

for p in bf.filter(products,green):
    print(f'-{p.name} is green')

    
print('Large Product (new):')
large=SizeSpecification(Size.Large)

for p in bf.filter(products,large):
    print(f'-{p.name} is large')
    
print('Large Blue Product (new):')
large_blue=AndSpecification(large,ColorSpecification(Color.Blue))

for p in bf.filter(products,large_blue):
    print(f'-{p.name} is large and blue')
    
print('Large Blue Product (new__and):')
large_blue=large & ColorSpecification(Color.Blue)

for p in bf.filter(products,large_blue):
    print(f'-{p.name} is large and blue')

Green Product (old):
-Apple is green
-Tree is green
Green Product (new):
-Apple is green
-Tree is green
Large Product (new):
-Tree is large
-House is large
Large Blue Product (new):
-House is large and blue
Large Blue Product (new__and):
-House is large and blue


## Liskov Substitution Principle

if you have some interface that takes some kind of base class then you can put a derived class and then everthing should work.

In [44]:
class Rectangle:
    def __init__(self, width, height):
        self._height = height
        self._width = width

    @property
    def area(self):
        return self._width * self._height

    def __str__(self):
        return f'Width: {self.width}, height: {self.height}'

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        self._width = value

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        self._height = value


def use_it(rc):
    w = rc.width
    rc.height = 10  # unpleasant side effect
    expected = int(w * 10)
    print(f'Expected an area of {expected}, got {rc.area}')


rc = Rectangle(2, 3)
use_it(rc)


Expected an area of 20, got 20


In [47]:
class Square(Rectangle):
    def __init__(self, size):
        Rectangle.__init__(self, size, size)

    @Rectangle.width.setter
    def width(self, value):
        _width = _height = value

    @Rectangle.height.setter
    def height(self, value):
        _width = _height = value


def use_it(rc):
    w = rc.width
    rc.height = 10  # unpleasant side effect
    expected = int(w * 10)
    print(f'Expected an area of {expected}, got {rc.area}')


rc = Rectangle(2, 3)
use_it(rc)

sq = Square(5)
use_it(sq)

Expected an area of 20, got 20
Expected an area of 50, got 25


## Interface Segregation Principle

Not to put too may interfaces 

In [49]:
from abc import abstractmethod


class Machine:
    def print(self, document):
        raise NotImplementedError()

    def fax(self, document):
        raise NotImplementedError()

    def scan(self, document):
        raise NotImplementedError()


# ok if you need a multifunction device
class MultiFunctionPrinter(Machine):
    def print(self, document):
        pass

    def fax(self, document):
        pass

    def scan(self, document):
        pass

#Problem is the old printer doesnot have fax and scanner, a instance will have the fax and scan.
class OldFashionedPrinter(Machine):
    def print(self, document):
        # ok - print stuff
        pass

    def fax(self, document):
        pass  # do-nothing

    def scan(self, document):
        """Not supported!"""
        raise NotImplementedError('Printer cannot scan!')

printer = OldFashionedPrinter()
printer.fax(123)  # nothing happens
printer.scan(123)  # oops!

## Here the client will fail , if it calls the scnner and fax for the oldfalhsioned printer. Which we can avoid by doing the below.

NotImplementedError: Printer cannot scan!

In [50]:
from abc import abstractmethod

## Instead of having the Machine Class have three different classes 
class Printer:
    @abstractmethod
    def print(self, document): pass


class Scanner:
    @abstractmethod
    def scan(self, document): pass


# same for Fax, etc.

class MyPrinter(Printer):
    def print(self, document):
        print(document)


class Photocopier(Printer, Scanner):
    def print(self, document):
        print(document)

    def scan(self, document):
        pass  # something meaningful


class MultiFunctionDevice(Printer, Scanner):  # , Fax, etc
    @abstractmethod
    def print(self, document):
        pass

    @abstractmethod
    def scan(self, document):
        pass


class MultiFunctionMachine(MultiFunctionDevice):
    def __init__(self, printer, scanner):
        self.printer = printer
        self.scanner = scanner

    def print(self, document):
        self.printer.print(document)

    def scan(self, document):
        self.scanner.scan(document)


## Dependency Inversion Principle

High level classes and high level modules shold not dedpend on low level classes and modules 

In [52]:
from abc import abstractmethod
from enum import Enum


class Relationship(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2


class Person:
    def __init__(self, name):
        self.name = name


class RelationshipBrowser:
    @abstractmethod
    def find_all_children_of(self, name): pass


class Relationships(RelationshipBrowser):  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))
            
    def find_all_children_of(self, name):
        for r in self.relations:
            if r[0].name == name and r[1] == Relationship.PARENT:
                yield r[2].name

                
class Relationships(RelationshipBrowser):  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))
            
    def find_all_children_of(self, name):
        for r in self.relations:
            if r[0].name == name and r[1] == Relationship.PARENT:
                yield r[2].name
                
# changed the init in the research to not have low level dependency , if the stroage type relations =[] is chnaged from list to dictonary the client doesnot have to do an change.
class Research:
    # dependency on a low-level module directly
    # bad because strongly dependent on e.g. storage type

    # def __init__(self, relationships):
    #     # high-level: find all of john's children
    #     relations = relationships.relations
    #     for r in relations:
    #         if r[0].name == 'John' and r[1] == Relationship.PARENT:
    #             print(f'John has a child called {r[2].name}.')

    def __init__(self, browser):
        for p in browser.find_all_children_of("John"):
            print(f'John has a child called {p}')


parent = Person('John')
child1 = Person('Chris')
child2 = Person('Matt')

# low-level module
relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

John has a child called Chris
John has a child called Matt


## Summary

### Single Responsibilty Principle :

    - A Class should only have one reason to change.
    - Seperation of concerns - different classes handling diffrent, independent tasks/problems

### Open-Closed Principle :

    - Classes should be open for extension but closed for modification

### Liskov Substituion Principle

    -You Should be able to substitue a basetype for a subtype

###  Interface Segregation Principle

    - Don't put too much into an interface ; split into a seperate interfaces
    - YAGNI - ou Ain't Going to Need It

### Dependency Inversion Principle

    - High-Level modules should not depend upon low-level ones; use abstractions